In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import mibian
from scipy.stats import norm
from datetime import datetime

# Black-Scholes function for put options price calculation
def black_scholes_put_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Load tickers from CSV file
csv_path = '/Users/Allison/Desktop/portfolio10.csv'
tickers_df = pd.read_csv(csv_path)

# Fetch the 10-year Treasury yield for the risk-free rate
treasury_ticker = yf.Ticker("^TNX")
risk_free_rate = treasury_ticker.history(period="1d")['Close'].iloc[0] / 100  # Convert to decimal

# Prepare DataFrames to store put option prices and Greeks
all_puts_data = pd.DataFrame()
all_puts_greeks = pd.DataFrame()

# Loop through each ticker in the portfolio
for ticker_symbol in tickers_df['ticker']:
    try:
        # Fetch stock data and options chain from yfinance
        ticker = yf.Ticker(ticker_symbol)
        current_price = ticker.history(period="1d")['Close'].iloc[0]

        # Retrieve options chain for each expiration date
        expiry_dates = ticker.options
        for expiry_date in expiry_dates:
            try:
                # Get put options for the specified expiration date
                options_chain = ticker.option_chain(expiry_date)
                puts = options_chain.puts

                # Calculate time to expiration in days and years
                expiration_date = datetime.strptime(expiry_date, "%Y-%m-%d")
                days_until_expiration = (expiration_date - datetime.now()).days
                time_to_expiration = days_until_expiration / 365

                # Loop through each put option to calculate the Black-Scholes price and Greeks
                for index, row in puts.iterrows():
                    strike_price = row['strike']
                    implied_volatility = row['impliedVolatility']

                    # Skip options with missing or zero implied volatility
                    if pd.isna(implied_volatility) or implied_volatility == 0:
                        continue

                    # Calculate Black-Scholes price
                    put_price = black_scholes_put_price(
                        S=current_price,
                        K=strike_price,
                        T=time_to_expiration,
                        r=risk_free_rate,
                        sigma=implied_volatility
                    )

                    # Calculate Greeks using Mibian
                    option = mibian.BS(
                        [current_price, strike_price, risk_free_rate, days_until_expiration],
                        volatility=implied_volatility * 100 p
                    )
                    delta = option.putDelta
                    gamma = option.gamma
                    theta = option.putTheta
                    vega = option.vega

                    # Store calculated price and Greeks
                    option_data = pd.DataFrame([{
                        "ticker": ticker_symbol,
                        "expiration_date": expiry_date,
                        "strike": strike_price,
                        "implied_volatility": implied_volatility,
                        "put_price": put_price,
                        "current_price": current_price,
                        "Delta": delta,
                        "Gamma": gamma,
                        "Theta": theta,
                        "Vega": vega
                    }])

                    # Append both price and Greeks to the main DataFrame
                    all_puts_data = pd.concat([all_puts_data, option_data], ignore_index=True)

            except Exception as e:
                print(f"Error processing options for {ticker_symbol} on expiry {expiry_date}: {e}")
                continue

    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")

# Filter options where the put price is less than the underlying price
affordable_puts = all_puts_data[all_puts_data['put_price'] < all_puts_data['current_price']]

# Display results
print("All Put Options Data with Greeks:")
print(all_puts_data)

print("\nAffordable Put Options (Put Price < Underlying Price):")
print(affordable_puts)

All Put Options Data with Greeks:
     ticker expiration_date  strike  implied_volatility  put_price  \
0      EXPE      2024-11-08    95.0            2.541996   0.318117   
1      EXPE      2024-11-08   100.0            2.328129   0.325155   
2      EXPE      2024-11-08   110.0            1.989258   0.408463   
3      EXPE      2024-11-08   115.0            1.686525   0.298151   
4      EXPE      2024-11-08   120.0            1.167973   0.057254   
...     ...             ...     ...                 ...        ...   
2869     HD      2027-01-15   400.0            0.203446  32.199225   
2870     HD      2027-01-15   410.0            0.203240  36.465769   
2871     HD      2027-01-15   420.0            0.201271  40.626065   
2872     HD      2027-01-15   450.0            0.180359  51.248900   
2873     HD      2027-01-15   460.0            0.174698  55.649485   

      current_price     Delta     Gamma     Theta      Vega  
0        159.399994 -0.018858  0.001086 -0.244215  0.007686  
1

In [6]:
# Filter for realistic implied volatility and Delta range
filtered_puts = affordable_puts[
    (affordable_puts['implied_volatility'] > 0.05) &  # Exclude very low volatility
    (affordable_puts['Delta'] >= -0.9) & (affordable_puts['Delta'] <= -0.7)  # Limit Delta range
]

# Initialize an empty DataFrame to store the best put option for each ticker
best_puts = pd.DataFrame()

# Loop through each unique ticker to choose the best option
for ticker in filtered_puts['ticker'].unique():
    ticker_data = filtered_puts[filtered_puts['ticker'] == ticker]
    ticker_data = ticker_data.sort_values(by=['Delta', 'Theta'], ascending=[True, True])
    best_option = ticker_data.iloc[0]
    best_puts = pd.concat([best_puts, pd.DataFrame([best_option])], ignore_index=True)

print("Refined Best Put Options for Each Ticker:")
print(best_puts)

Refined Best Put Options for Each Ticker:
  ticker expiration_date  strike  implied_volatility  put_price  \
0   EXPE      2024-11-08   177.5            0.950928  19.219410   
1     GM      2024-11-08    54.0            0.424810   3.122142   
2    USB      2024-11-15    51.0            0.304694   3.244211   
3    ACN      2025-01-17   390.0            0.207772  42.832849   
4    FIS      2025-01-17   100.0            0.232430  10.810332   
5    AXP      2024-11-15   300.0            0.418219  27.867313   
6   EBAY      2027-01-15    85.0            0.181893  20.468627   
7    AMT      2024-11-15   230.0            0.373541  19.101491   
8    MAR      2024-12-20   290.0            0.265510  29.694440   
9     HD      2025-03-21   490.0            0.265327  92.929476   

   current_price     Delta     Gamma     Theta      Vega  
0     159.399994 -0.848637  0.014783 -0.465078  0.039142  
1      50.959999 -0.899831  0.077535 -0.049718  0.009374  
2      47.840000 -0.881508  0.078349 -0.022